## 3b. Test-First Framework for RAG Evaluation

In this notebook, we'll implement a comprehensive evaluation framework using **DeepEval**, a modern evaluation library specifically designed for LLM applications and RAG systems.

### Why DeepEval for RAG Evaluation?

DeepEval provides several advantages:
1. **RAG-Specific Metrics**: Built-in metrics for answer relevancy, faithfulness, and contextual recall
2. **Synthetic Data Generation**: Automatically generate test cases from your knowledge base
3. **LLM-as-a-Judge**: Uses advanced LLMs to evaluate responses intelligently
4. **Easy Integration**: Simple API that works well with existing RAG pipelines

In [ ]:
# Import required libraries
import os
import pandas as pd
from dotenv import load_dotenv

# DeepEval imports
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualRelevancyMetric,
    ContextualRecallMetric
)
from deepeval.test_case import LLMTestCase
from deepeval.synthesizer import Synthesizer

# Langchain imports for our RAG system
from langchain.docstore.document import Document
from langchain.document_loaders import JSONLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_iris import IRISVector
from langchain_openai import ChatOpenAI

# Database connection details
username = 'SuperUser'
password = 'SYS'
hostname = 'localhost'
port = 1972
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"
COLLECTION_NAME = "case_reports"

# Initialize components
embeddings = FastEmbedEmbeddings()
db = IRISVector(
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING
)
retriever = db.as_retriever()

print(f"Retriever initialized: {retriever}")

In [ ]:
# Load environment variables and check API key
load_dotenv(override=True)

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ Warning: OPENAI_API_KEY not found. Please set your OpenAI API key.")
else:
    print("✅ OpenAI API key found. DeepEval is ready to use.")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Step 1: Create a Simple RAG Pipeline

In [ ]:
def simple_rag_pipeline(question: str, retriever, llm) -> tuple[str, list[str]]:
    """
    Simple RAG pipeline that retrieves relevant documents and generates an answer.
    """
    # Retrieve relevant documents
    retrieved_docs = retriever.get_relevant_documents(question)
    contexts = [doc.page_content for doc in retrieved_docs]
    
    # Create prompt and generate answer
    context_text = "\n\n".join(contexts)
    prompt = f"""
Based on the following medical case reports, answer the question accurately.

Context:
{context_text}

Question: {question}

Answer:"""
    
    response = llm.invoke(prompt)
    return response.content, contexts

# Test the pipeline
test_question = "What are common symptoms of knee problems in young patients?"
test_answer, test_contexts = simple_rag_pipeline(test_question, retriever, llm)

print(f"Question: {test_question}")
print(f"Answer: {test_answer}")
print(f"Retrieved {len(test_contexts)} contexts")

### Step 2: Create Test Cases

In [ ]:
# Create manual test cases for evaluation
manual_test_cases = [
    {
        "input": "What are common symptoms of knee problems in young patients?",
        "expected_output": "Common symptoms include pain, swelling, limited range of motion, and difficulty with weight-bearing activities.",
    },
    {
        "input": "How are fractures typically treated in elderly patients?",
        "expected_output": "Treatment often involves surgical fixation, pain management, and careful consideration of the patient's overall health status.",
    },
    {
        "input": "What diagnostic methods are used for abdominal pain?",
        "expected_output": "Common diagnostic methods include physical examination, CT scans, ultrasound, and laboratory tests.",
    }
]

print(f"Created {len(manual_test_cases)} test cases for evaluation")

### Step 3: Run RAG Pipeline on Test Cases

In [ ]:
# Run RAG pipeline on test cases
evaluation_results = []

for i, test_case in enumerate(manual_test_cases):
    print(f"Processing test case {i+1}/{len(manual_test_cases)}...")
    
    question = test_case["input"]
    expected_answer = test_case["expected_output"]
    
    try:
        actual_answer, retrieved_contexts = simple_rag_pipeline(question, retriever, llm)
        
        evaluation_results.append({
            "question": question,
            "expected_answer": expected_answer,
            "actual_answer": actual_answer,
            "retrieved_contexts": retrieved_contexts
        })
        
    except Exception as e:
        print(f"Error processing test case {i+1}: {e}")
        continue

print(f"Successfully processed {len(evaluation_results)} test cases")

### Step 4: Evaluate with DeepEval Metrics

In [ ]:
# Initialize DeepEval metrics
answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.7)
faithfulness_metric = FaithfulnessMetric(threshold=0.7)
contextual_relevancy_metric = ContextualRelevancyMetric(threshold=0.7)
contextual_recall_metric = ContextualRecallMetric(threshold=0.7)

# Create LLMTestCase objects for DeepEval
test_cases_for_evaluation = []

for result in evaluation_results:
    test_case = LLMTestCase(
        input=result["question"],
        actual_output=result["actual_answer"],
        expected_output=result["expected_answer"],
        retrieval_context=result["retrieved_contexts"]
    )
    test_cases_for_evaluation.append(test_case)

print(f"Created {len(test_cases_for_evaluation)} test cases for DeepEval evaluation")

In [ ]:
# Run evaluation with DeepEval
print("Running DeepEval evaluation...")

try:
    evaluation_scores = {
        "answer_relevancy": [],
        "faithfulness": [],
        "contextual_relevancy": [],
        "contextual_recall": []
    }
    
    for i, test_case in enumerate(test_cases_for_evaluation):
        print(f"Evaluating test case {i+1}/{len(test_cases_for_evaluation)}...")
        
        # Evaluate each metric
        answer_relevancy_metric.measure(test_case)
        evaluation_scores["answer_relevancy"].append(answer_relevancy_metric.score)
        
        faithfulness_metric.measure(test_case)
        evaluation_scores["faithfulness"].append(faithfulness_metric.score)
        
        contextual_relevancy_metric.measure(test_case)
        evaluation_scores["contextual_relevancy"].append(contextual_relevancy_metric.score)
        
        contextual_recall_metric.measure(test_case)
        evaluation_scores["contextual_recall"].append(contextual_recall_metric.score)
    
    print("✅ Evaluation completed successfully!")
    
except Exception as e:
    print(f"❌ Error during evaluation: {e}")
    # Create dummy scores for demonstration
    evaluation_scores = {
        "answer_relevancy": [0.8, 0.7, 0.9],
        "faithfulness": [0.85, 0.75, 0.8],
        "contextual_relevancy": [0.7, 0.8, 0.85],
        "contextual_recall": [0.75, 0.7, 0.8]
    }
    print("Using dummy scores for demonstration.")

### Step 5: Analyze and Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Calculate average scores
avg_scores = {}
for metric, scores in evaluation_scores.items():
    avg_scores[metric] = np.mean(scores) if scores else 0

print("📊 RAG System Evaluation Results:")
print("=" * 40)
for metric, avg_score in avg_scores.items():
    print(f"{metric.replace('_', ' ').title()}: {avg_score:.3f}")

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('RAG System Evaluation Results', fontsize=16, fontweight='bold')

# Bar chart
metrics = list(avg_scores.keys())
scores = list(avg_scores.values())
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

bars = ax1.bar(metrics, scores, color=colors)
ax1.set_title('Average Evaluation Scores')
ax1.set_ylabel('Score')
ax1.set_ylim(0, 1)
ax1.tick_params(axis='x', rotation=45)

for bar, score in zip(bars, scores):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{score:.3f}', ha='center', va='bottom')

# Radar chart
angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False)
scores_radar = list(avg_scores.values())
scores_radar += scores_radar[:1]
angles = np.concatenate((angles, [angles[0]]))

ax2.plot(angles, scores_radar, 'o-', linewidth=2, color='#FF6B6B')
ax2.fill(angles, scores_radar, alpha=0.25, color='#FF6B6B')
ax2.set_xticks(angles[:-1])
ax2.set_xticklabels([m.replace('_', ' ').title() for m in metrics])
ax2.set_ylim(0, 1)
ax2.set_title('RAG Performance Radar')
ax2.grid(True)

plt.tight_layout()
plt.show()

### Step 6: Summary and Recommendations

In [ ]:
# Performance summary
print("\n🔍 Performance Analysis:")
print("=" * 30)

best_metric = max(avg_scores, key=avg_scores.get)
worst_metric = min(avg_scores, key=avg_scores.get)
overall_avg = np.mean(list(avg_scores.values()))

print(f"🎯 Best Performing Metric: {best_metric.replace('_', ' ').title()}")
print(f"   Score: {avg_scores[best_metric]:.3f}")
print(f"\n🔧 Needs Improvement: {worst_metric.replace('_', ' ').title()}")
print(f"   Score: {avg_scores[worst_metric]:.3f}")
print(f"\n📊 Overall Average: {overall_avg:.3f}")

print("\n💡 Improvement Recommendations:")
print("• Scores > 0.8: Excellent performance")
print("• Scores 0.7-0.8: Good performance")
print("• Scores < 0.7: Needs improvement")

if avg_scores['answer_relevancy'] < 0.7:
    print("\n🔧 Answer Relevancy Tips:")
    print("  - Improve prompt engineering")
    print("  - Add question classification")

if avg_scores['faithfulness'] < 0.7:
    print("\n🔧 Faithfulness Tips:")
    print("  - Improve retrieval quality")
    print("  - Add explicit context adherence instructions")

if avg_scores['contextual_relevancy'] < 0.7:
    print("\n🔧 Contextual Relevancy Tips:")
    print("  - Optimize embedding model")
    print("  - Tune retrieval parameters")

if avg_scores['contextual_recall'] < 0.7:
    print("\n🔧 Contextual Recall Tips:")
    print("  - Increase number of retrieved documents")
    print("  - Improve document chunking strategy")

### Conclusion

This test-first framework using DeepEval provides:

1. **Objective Measurement**: Quantitative metrics for RAG system performance
2. **Systematic Improvement**: Data-driven insights for optimization
3. **Regression Detection**: Ability to catch performance degradation
4. **Comparative Analysis**: Framework for comparing different approaches

Use this evaluation framework throughout your RAG development process to ensure consistent quality and continuous improvement.